---
# Módulo 4 - Estatística para MF
---

In [1]:
import numpy as np
import pandas as pd

import pylab
import statsmodels.api as sm
from sklearn import linear_model
from scipy import stats
from sklearn.linear_model import LinearRegression
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import yfinance as yf
import vectorbt as vbt

import seaborn as sns
import plotly.graph_objects as go
import plotly_express as px
import sweetviz as sv
from plotly.subplots import make_subplots

## 1. **Distribuição dos dados**

Distribuição dos dados é o ponto chave para praticamente tudo o que entendemos por estatística

Apesar disso, muitas vezes é simplesmente ignorada nos modelos. Muitas vezes, modelos podem \
não funcionar por causa de algumas poucas observações que distorcem a distribuição dos dados.

### 1.1. Distribuição Normal

In [ ]:
normal_dist = np.random.normal(0, 1, 1000)
normal_fig1 = sns.histplot(normal_dist, kde=True)

In [ ]:
print(np.mean(normal_dist))
print(np.median(normal_dist))
print(stats.mode(normal_dist))

In [ ]:
fig01 = px.histogram(normal_dist, color_discrete_sequence=["lightseagreen"])
fig01.add_vline(x=np.mean(normal_dist), line_width=3, line_color="red")
fig01.add_vline(
    x=np.median(normal_dist), line_width=3, line_dash="dash", line_color="red"
)
fig01.update_layout(
    width=600,
    height=600,
    template="simple_white",
    paper_bgcolor="#f7f8fa",
    margin=dict(l=20, r=20, t=20, b=20),
    showlegend=False,
)

#### Testes de Normalidade

- À medida que aumenta o uso de grandes volumes de dados, os testes de normalidade vão caindo em desuso, pois eles são sensíveis justamente à quantidade de dados.


In [ ]:
stats.normaltest(normal_dist)

## pvalue > 0.05 --> normal

#### Medidas de posição: quantis

Quantis são pontos de corte que dividem um intervalo de distribuição de probabilidade em intervalos contínuos com probabilidades iguais, ou as observações de uma amostra do mesmo jeito.

<img src="https://cdn.scribbr.com/wp-content/uploads/2022/05/Quartiles-probability-distribution.webp" width="800" align="left"/>

#### Quantile - Quantile PLot (Q-Q Plot)

Compara os quanits do modelo real (imperfeito) com um modelo teórico (perfeitamente normal)

In [ ]:
stats.probplot(normal_dist, dist="norm", plot=pylab)
pylab.show()

In [ ]:
fig02 = go.Figure()
fig02.add_trace(
    go.Box(
        y=normal_dist,
        boxpoints="all",
        boxmean="sd",
        name="<b>Normal distribution",
        marker_color="blue",
    )
)
fig02.update_layout(
    width=450,
    height=600,
    template="simple_white",
    paper_bgcolor="#f7f8fa",
    margin=dict(l=20, r=20, t=20, b=20),
    showlegend=False,
)

print(np.quantile(normal_dist, 0.25))
print(np.percentile(normal_dist, 25))
fig02.show()

In [ ]:
pd.DataFrame(normal_dist).describe()

#### Skewness


<img src="https://www.biologyforlife.com/uploads/2/2/3/9/22392738/c101b0da6ea1a0dab31f80d9963b0368_orig.png" width="800" align="left"/>

In [ ]:
stats.skew(normal_dist)

#### Medidas de dispersão

> Amplitude, desvio-padrão e médio, coef de variação e int de confiança 

In [ ]:
# Desvio Padrão
# Sharpe ratio, bandas de bollinger

np.std(normal_dist, ddof=True)

In [ ]:
# Coef de Variação
# Comparar e avaliar a consistência (ex, pagamento de dividendos)

np.std(normal_dist, ddof=True) / np.mean(normal_dist) * 100

In [ ]:
# Erro padrão da Média

stats.stats.sem(normal_dist)

In [ ]:
# Intervalo de Confiança

ic_95 = stats.t.interval(
    confidence=0.95,
    df=len(normal_dist) - 1,
    loc=np.mean(normal_dist),
    scale=stats.sem(normal_dist),
)
ic_95

#### Curtose

<img src="https://www.researchgate.net/profile/Attila-Bonyar/publication/298415862/figure/fig1/AS:340236723867648@1458130164255/Illustration-of-the-skewness-and-kurtosis-values-and-how-they-correlate-with-the-shape-of_W640.jpg" width="600" align="left"/>

In [ ]:
stats.kurtosis(normal_dist, fisher=True)

In [ ]:
# Variáveis discretas

## 2. **Estudo de caso**

### MGLU3

In [ ]:
mglu3 = yf.download("MGLU3.SA", period="max")["Adj Close"]

In [ ]:
go.Histogram(x=mglu3)

In [ ]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Histogram(x=mglu3), row=1, col=2)
fig.add_vline(x=np.mean(mglu3), line_width=3, line_color="red", row=1, col=2)
fig.add_vline(
    x=np.median(mglu3), line_width=3, line_dash="dash", line_color="red", row=1, col=2
)

fig.add_trace(go.Box(y=mglu3, boxpoints="all", boxmean="sd"), row=1, col=1)

fig.update_layout(
    width=800, height=600, template="simple_white", margin=dict(l=20, r=20, t=20, b=20)
)

### IBOV

In [ ]:
ibov = yf.download("^BVSP", start="2000-01-01")["Adj Close"]

In [ ]:
fig = px.histogram(ibov, color_discrete_sequence=["lightseagreen"], nbins=100)
fig.add_vline(x=np.mean(ibov), line_width=3, line_color="black")
fig.add_vline(x=np.median(ibov), line_width=3, line_dash="dash", line_color="red")

fig.update_layout(
    width=1000,
    height=600,
    template="simple_white",
    margin=dict(l=5, r=5, t=5, b=5),
    showlegend=False,
)
print("Fechamento")
fig.show()
print("Distribuição Trimodal, não reprensenta uma curva normal")

In [ ]:
stats.probplot(ibov, dist="norm", plot=pylab)
pylab.show()

In [ ]:
retorno = ibov.pct_change() * 100

In [ ]:
fig = px.histogram(retorno, color_discrete_sequence=["lightseagreen"])



fig.add_vline(x=np.mean(retorno), line_width=3, line_color="black")
fig.add_vline(x=retorno.median(), line_width=3, line_dash="dash", line_color="red")

fig.update_layout(
    width=1000,
    height=600,
    template="simple_white",
    margin=dict(l=5, r=5, t=20, b=5),
    showlegend=False,
)
print("Retorno")
fig.show()

In [ ]:
stats.probplot(retorno, dist="norm", plot=pylab)
pylab.show()

### DY Petro

In [ ]:
div_petro = vbt.YFData.download("PETR4.SA").get(["Close", "Dividends"])

In [ ]:
div_petro["soma_div"] = div_petro.Dividends.rolling("365D").sum()

In [ ]:
div_petro["DY"] = div_petro["soma_div"] / div_petro["Close"] * 100

In [ ]:
dy_petro = div_petro[div_petro.Dividends != 0]

In [ ]:
sns.histplot(dy_petro.DY, kde=True)

In [ ]:
fig = px.scatter(x=dy_petro.Close, y=dy_petro.DY)
fig.update_layout(
    width=600,
    height=600,
    template="simple_white",
    margin=dict(l=5, r=20, t=20, b=5),
    xaxis_title="<b>Price",
    yaxis_title="<b>Dividend Yield",
)

Correlação (Peason)

In [ ]:
corr, p = stats.pearsonr(dy_petro.Close, dy_petro.DY)
print("r = %.3f" % corr, " p = %.3f" % p)

Correlação (Spearman) \
- Não-paramétrica e não assume distribuição normal

In [ ]:
corr, p = stats.spearmanr(dy_petro.Close, dy_petro.DY)
print("r = %.3f" % corr, " p = %.3f" % p)

### Arbitragem em ações

In [ ]:
petros = yf.download(["PETR4.SA", "PETR3.SA"], start="2010-01-04")["Adj Close"]

In [ ]:
petros["Spread"] = (petros["PETR4.SA"] / petros["PETR3.SA"]).round(3)

In [ ]:
# Métricas
std = np.std(petros.Spread)
mean_spread = round(np.mean(petros.Spread), 3)
sd1_min_spread = mean_spread - round(std, 3)
sd1_max_spread = mean_spread + round(std, 3)
sd2_min_spread = mean_spread - (2 * round(std, 3))
sd2_max_spread = mean_spread + (2 * round(std, 3))

In [ ]:
fig = px.line(petros, x=petros.index, y=petros.Spread)

fig.add_hline(y=mean_spread, line_width=3, line_color="red")
fig.add_hline(y=sd1_min_spread, line_width=3, line_color="green", line_dash="dot")
fig.add_hline(y=sd1_max_spread, line_width=3, line_color="green", line_dash="dot")
fig.add_hline(y=sd2_min_spread, line_width=3, line_color="orange", line_dash="dash")
fig.add_hline(y=sd2_max_spread, line_width=3, line_color="orange", line_dash="dash")

fig.update_layout(
    width=1500,
    height=600,
    xaxis_rangeslider_visible=False,
    title_text="Razão entre preço PETR4 e PETR3 (2010 a jul/2024)",
    margin=dict(l=20, r=20, t=70, b=20),
)
fig

## 3. **Médias Móveis**


In [ ]:
# Suavização de Curvas

petros["MM50"] = petros["PETR4.SA"].rolling(50).mean()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=petros.index, y=petros["PETR4.SA"], name="PETR4.SA"))
fig.add_trace(go.Scatter(x=petros.index, y=petros["MM50"], name="MM 50"))
fig.add_trace(go.Scatter(x=petros.index, y=petros["MM20"], name="MM 20"))

fig.update_layout(
    width=1200,
    height=600,
    xaxis_rangeslider_visible=False,
    title_text="Suavização da Curva de preções PETR4",
    margin=dict(l=10, r=20, b=20, t=60),
)

### Z Score

In [ ]:
petros['ic95_mm20_lower'] = petros['PETR4.SA'].rolling(20).mean() - 1.96 * petros['PETR4.SA'].rolling(20).std()
petros['ic95_mm20_upper'] = petros['PETR4.SA'].rolling(20).mean() + 1.96 * petros['PETR4.SA'].rolling(20).std()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=petros.index, y=petros["PETR4.SA"], name="PETR4.SA"))
fig.add_trace(go.Scatter(x=petros.index, y=petros["MM20"], name="MM 20"))
fig.add_trace(go.Scatter(x=petros.index, y=petros["ic95_mm20_lower"], name="IC 95 lower MM 20"))
fig.add_trace(go.Scatter(x=petros.index, y=petros["ic95_mm20_upper"], name="IC 95 upper MM 20"))

fig.update_layout(
    width=1200,
    height=600,
    xaxis_rangeslider_visible=False,
    title_text="Suavização da Curva de preções PETR4",
    margin=dict(l=10, r=20, b=20, t=60),
)

## Sweetviz

In [ ]:
ticker_IBOV = ['VALE3', 'PETR4', 'WEGE3', 'ITUB4', 'MGLU3',
               'BBDC4', 'ABEV3', 'BBAS3', 'ELET3', 'RENT3',
               'ITSA4', 'JBSS3', 'SUZB3', 'LREN3', 'EQTL3']

In [ ]:
ticker_IBOV = [x + '.SA' for x in ticker_IBOV]

In [ ]:
ticker_IBOV.extend(['^BVSP', 'BRL=x'])

In [ ]:
data_inicio = '2012-08-23'
data_fim = '2023-02-01'

In [ ]:
df = yf.download(ticker_IBOV, start=data_inicio, end=data_fim)['Adj Close']

In [ ]:
view = sv.analyze(df)

In [ ]:
view.show_html('Relatorio.html')

## 4. **Tranformações de dados**


### 4.1 Logarítimica

* Comparação de escalas/distribuições diferentes
* Permite trabalhar com dados não normais
* Avaliar relações não-lineares
* Séries temporais longas

In [ ]:
ibov_sp500 = yf.download(['^BVSP','^GSPC'], period='max')['Close']

In [ ]:
def graph01():
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=ibov_sp500.index, y=ibov_sp500['^GSPC'],
                            name='S&P 500', line=dict(color='blue', width=3)))

    fig.add_vrect(x0='1973', x1='1985', fillcolor="#000000", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['1979'],y=[1800],mode="text",text="<b>Petróleo e",textposition="top center",showlegend=False))
    fig.add_trace(go.Scatter(x=['1979'],y=[1500],mode="text",text="<b>Energia",textposition="top center",showlegend=False))

    fig.add_vrect(x0='2000-06', x1='2003-06', fillcolor="blue", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['2002-06'],y=[3000],mode="text",text="<b>Bolha da",textposition="top center",showlegend=False))
    fig.add_trace(go.Scatter(x=['2002-06'],y=[2700],mode="text",text="<b>Internet",textposition="top center",showlegend=False))

    fig.add_vrect(x0='2008', x1='2013', fillcolor="green", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['2010-06'],y=[4000],mode="text",text="<b>Crise",textposition="top center",showlegend=False))
    fig.add_trace(go.Scatter(x=['2010-06'],y=[3700],mode="text",text="<b>Subprimes",textposition="top center",showlegend=False))

    fig.add_vrect(x0='2020', x1='2021', fillcolor="red", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['2021'],y=[4000],mode="text",text="<b>COVID-19",textposition="top center",showlegend=False))

    fig.update_layout(title='', xaxis_title="<b>Data", yaxis_title="<b>Valor nominal (USD)", legend_title="",
                    font=dict(family="Arial, Arial, Arial",size=20,color="black"),paper_bgcolor="#f7f8fa")

    fig.update_layout(xaxis_rangeslider_visible=False, margin=dict(l=120, r=20, t=20, b=20), template = 'simple_white',
                    width=1000,height=500,legend=dict(orientation="h",yanchor="bottom",y=0.9,xanchor="right",x=0.15),
                    xaxis=dict(range=['1950', '2025']))

    fig.show()
graph01()

In [ ]:
ibov_sp500['^GSPC'].vbt.drawdowns.plot(width=1000).show()

In [ ]:
def graph02():
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=ibov_sp500.index, y=np.log(ibov_sp500['^GSPC']),
                            name='S&P 500', line=dict(color='blue', width=3)))

    fig.add_vrect(x0='1973', x1='1985', fillcolor="#000000", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['1979'],y=np.log([1800]),mode="text",text="<b>Petróleo e",textposition="top center",showlegend=False))
    fig.add_trace(go.Scatter(x=['1979'],y=np.log([1100]),mode="text",text="<b>Energia",textposition="top center",showlegend=False))

    fig.add_vrect(x0='2000-06', x1='2003-06', fillcolor="blue", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['2002-06'],y=np.log([3000]),mode="text",text="<b>Bolha da",textposition="top center",showlegend=False))
    fig.add_trace(go.Scatter(x=['2002-06'],y=np.log([2100]),mode="text",text="<b>Internet",textposition="top center",showlegend=False))

    fig.add_vrect(x0='2008', x1='2013', fillcolor="green", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['2010-06'],y=np.log([4000]),mode="text",text="<b>Crise",textposition="top center",showlegend=False))
    fig.add_trace(go.Scatter(x=['2010-06'],y=np.log([3100]),mode="text",text="<b>Subprimes",textposition="top center",showlegend=False))

    fig.add_vrect(x0='2020', x1='2021', fillcolor="red", opacity=0.25, line_width=0)
    fig.add_trace(go.Scatter(x=['2021'],y=np.log([4000]),mode="text",text="<b>COVID-19",textposition="top center",showlegend=False))

    fig.update_layout(title='', xaxis_title="<b>Data", yaxis_title="<b>Valor nominal (USD)", legend_title="",
                    font=dict(family="Arial, Arial, Arial",size=20,color="black"),paper_bgcolor="#f7f8fa")

    fig.update_layout(xaxis_rangeslider_visible=False, margin=dict(l=120, r=20, t=20, b=20), template = 'simple_white',
                    width=1000,height=500,legend=dict(orientation="h",yanchor="bottom",y=0.9,xanchor="right",x=0.15),
                    xaxis=dict(range=['1950', '2025']))

    fig.show()
graph02()

### 4.2 Normalização

Transforma os dados para um mesmo ponto de partida. Utilizado para comparar duas variáveis com diferentes magnitudes

In [ ]:
tickers = yf.download(['^BVSP', '^GSPC'], period = 'max')['Close']

In [ ]:
tickers = tickers.dropna()
tickers = tickers[tickers.index >='2000']

tickers.plot()

In [ ]:
tickers = tickers/tickers.iloc[0]
tickers.plot()

### 4.3 Scale 

In [ ]:
ativos = ['VALE3.SA', 'PETR4.SA', 'ITUB4.SA']

In [ ]:
vol = vbt.YFData.download(ativos, start='5 year ago', interval = '1d').get('Volume')

In [ ]:
vol.head()

In [ ]:
vol.vbt.histplot().show()

Quando as escalas são muito diferentes, a padronização dos dados é uma ótima forma pra compará-los
A padronização (scale) transforma os dados de diferentes grandezas para MÉDIA = 0 e VARIÂNCIA = 1

In [ ]:
vol_scale = vol.vbt.scale()

vol_scale.vbt.histplot().show()

### 4.4 Min Max

MINMAX é outra forma de transformação é baseado nos valores máximos e mínimos <p>
A padronização de mínimo e máximo (minmax_scale) transforma os dados de diferentes grandezas para MÍNIMO = 0 e MÁXIMO = 1

In [ ]:
vol.vbt.minmax_scale().head()

In [ ]:
vol.vbt.minmax_scale().plot()

### 4.5 Binarização

In [ ]:
milho = yf.download('ZC=F', start='2010-01-01', interval='1d')['Close']

Binariza os dados utilizando um threshold. Uma especie de filtro

In [ ]:
milho_pct = milho.pct_change()*100
milho_pct.dropna(inplace=True)

In [ ]:
milho_th_1 = milho_pct.vbt.binarize(threshold=1)
milho_th_1 = pd.DataFrame(milho_th_1)

In [ ]:
milho_th_1[milho_th_1.Close == 1]

#Todos os dias que o fechamento foi maior que 1%

## 5. Estatística Inferencial

A estatística é uma **ciência sobre a incerteza**: probabilidade de um fenômeno ocorrer, desde que satisfeitas um conjunto de condições teóricas

Fenômeno/observação comumente estão enviesados \
Baseado no nosso passado e experiências prévias, tendemos a chegar a conclusões distorcidas.

A estatística é uma **ciência sobre a incerteza**: probabilidade de um fenômeno ocorrer, desde que satisfeitas um conjunto de condições teóricas

Fenômeno/observação comumente estão enviesados


Baseado no nosso passado e experiências prévias, tendemos a chegar a conclusões distorcidas.

Na estatística inferencial, fazemos:

* Inferir sobre uma variável da população com base nos dados das amostras (repetições) - é muito difícil estudar a população inteira

* Generalizar uma observação a partir de probabilidades

### 5.1 Teste de Hipótese, p-valor e magnitude

Os **testes de hipótese** são amplamente utilizados em basicamente todas as áreas da ciência, incluindo a medicina, psicologia, finanças, marketing, entre outras, para testar teorias, avaliar o impacto de intervenções ou mudanças e fazer previsões.

Elas objetivam fornecer um método que permita verificar se os dados amostrais trazem evidências que apóiam ou não uma hipótese formulada.

São complementares aos mais variados testes estatísticos, desde os mais simples como test T, Anova, até os mais complexos, como modelos lineares mistos, logísticos...

O processo do teste de hipótese envolve duas hipóteses: a hipótese nula (H0), que é a afirmação a ser testada, e a hipótese alternativa (Ha), que é a afirmação oposta à hipótese nula.

Para realizar o teste, é escolhido um nível de significância (geralmente 5% ou 1%) que define o limite para a rejeição da hipótese nula. A partir da amostra, é calculado um valor chamado de estatística de teste, que é comparado a um valor crítico obtido a partir de uma tabela ou calculado através de métodos estatísticos.


---
**Vejamos na prática como funcionam os testes de hipótese?**

*Exemplo: Quero saber se as médias dos retornos do IBOV diferem entre os meses do ano*


**1° passo: determinar as hipóteses nula e alternativa**

* Hipótese H0 (nula): As médias dos retornos do IBOV <u>não diferem</u> entre os meses do ano, ou seja, são iguais estatisticamente
* Hipótese H1 (alternativa): As médias <u>diferem</u>, são diferentes significativamente

**2° passo: definir o nível de significância de teste**

<img src="https://miro.medium.com/max/1400/1*BwL6FDQ2B3QDKAwmcEVDSg.png" width="500" align="left"/>


**3° passo: executar o teste (seja ele qual for), checar o "valor de p" para decidir qual hipótese foi corroborada**

Se a estatística do teste é menor que o valor crítico, a hipótese nula não é rejeitada, o que significa que não há evidência suficiente para afirmar que a hipótese alternativa é verdadeira.

Por outro lado, se a estatística de teste é maior que o valor crítico, a hipótese nula é rejeitada e concluímos que a hipótese alternativa é provável.

### Valor de p

Vamos relembrar que no início do módulo que a estatística enxerga o mundo a partir de duas dimensões:

* Magnitude
* Probabilidade

O teste de hipótese vai se basear no p (probabilidade)


**p-valor**: associado ao nível de significância do teste. De forma pragmática, é uma medida que representa a probabilidade da hipótese nula ser verdadeira considerando o seu conjunto de dados.



> p = 0.05 indica que há uma chance em 1 em 20 de que a hipótese nula seja corroborada. Em outras palavras, 5% de chance de que a relação estudada (resultado do teste estatístico executado) seja devido ao acaso, e não devido a fatores que estão sendo estudados

O valor de "p" é um número entre 0 e 1 e é interpretado da seguinte forma:

* se o valor de p é menor que um nível de significância **pré-definido** (geralmente 0.05 ou 0.01), então a hipótese nula é rejeitada em favor da hipótese alternativa.

Por exemplo, se o valor de p for 0.03 e o nível de significância for 0.05, então a hipótese nula será rejeitada, o que significa que há evidências estatísticas para suportar a hipótese alternativa.
Quanto menor o valor de p, maior é a evidência contra a hipótese nula (**em termos de probabilidade, mas não de efeito**).

### Magnitude do efeito/relação/comparação

Além da probablidade de ocorrência, é importante checar o quão forte é o efeito/relação/comparação. Pode ser indicado por diversos parâmetros dependendo do teste, como os coeficiente de correlação (r), coeficiente de determinação (r²), diferença entre as médias (num caso de teste de comparação entre médias)...

Imagine que você encontrou que as médias de retorno mensal do ativo A são significativamente (p<0.05) maiores que as do ativo B, mas a diferença é de 0.02%

### Ánálises estatísticas em papers

**Efeito COVID-19 nos retornos dos principais índices mundiais**

https://koreascience.kr/article/JAKO202020952022559.pdf

**Efeito COVID-19 nos retornos dos principais índices mundiais**

https://koreascience.kr/article/JAKO202020952022559.pdf

## 6. **Comparação de médias entre grupos** (_Teste T e Anova_)

### 6.1 Cálculo do DY

In [2]:
ativos_setor_IBOV = {"AZUL4" : "Bens_industriais", "CCRO3" : "Bens_industriais", "ECOR3" : "Bens_industriais", "EMBR3" : "Bens_industriais",
                     "GOLL4" : "Bens_industriais", "RAIL3" : "Bens_industriais", "WEGE3" : "Bens_industriais",

                     "ALPA4" : "Consumo_ciclico", "ARZZ3" : "Consumo_ciclico", "CVCB3" : "Consumo_ciclico", "CYRE3" : "Consumo_ciclico",
                     "EZTC3" : "Consumo_ciclico", "LREN3" : "Consumo_ciclico", "MGLU3" : "Consumo_ciclico", "MRVE3" : "Consumo_ciclico",
                     "PETZ3" : "Consumo_ciclico", "SOMA3" : "Consumo_ciclico", "VIIA3" : "Consumo_ciclico",

                     "ABEV3" : "Consumo_nao_ciclico", "ASAI3" : "Consumo_nao_ciclico", "BEEF3" : "Consumo_nao_ciclico",
                     "BRFS3" : "Consumo_nao_ciclico", "CRFB3" : "Consumo_nao_ciclico", "JBSS3" : "Consumo_nao_ciclico",
                     "MRFG3" : "Consumo_nao_ciclico", "NTCO3" : "Consumo_nao_ciclico", "PCAR3" : "Consumo_nao_ciclico",
                     "RAIZ4" : "Consumo_nao_ciclico", "SLCE3" : "Consumo_nao_ciclico", "SMTO3" : "Consumo_nao_ciclico",

                     "ALSO3" : "Financeiro_e_outros", "B3SA3" : "Financeiro_e_outros", "BBAS3" : "Financeiro_e_outros",
                     "BBDC3" : "Financeiro_e_outros", "BBDC4" : "Financeiro_e_outros", "BBSE3" : "Financeiro_e_outros",
                     "BPAC11" : "Financeiro_e_outros", "BPAN4" : "Financeiro_e_outros", "CIEL3" : "Financeiro_e_outros",
                     "IGTI11" : "Financeiro_e_outros", "ITSA4" : "Financeiro_e_outros", "ITUB4" : "Financeiro_e_outros",
                     "MULT3" : "Financeiro_e_outros", "SANB11" : "Financeiro_e_outros",

                     "BRAP4" : "Mats_basicos", "BRKM5" : "Mats_basicos", "CMIN3" : "Mats_basicos", "CSNA3" : "Mats_basicos", "DXCO3" : "Mats_basicos",
                     "GGBR4" : "Mats_basicos", "GOAU4" : "Mats_basicos", "KLBN11" : "Mats_basicos", "SUZB3" : "Mats_basicos", "USIM5" : "Mats_basicos",
                     "VALE3" : "Mats_basicos",

                     "CSAN3" : "Petroleo_gas_biocombustiveis", "PETR3" : "Petroleo_gas_biocombustiveis", "PETR4" : "Petroleo_gas_biocombustiveis",
                     "PRIO3" : "Petroleo_gas_biocombustiveis", "RRRP3" : "Petroleo_gas_biocombustiveis", "UGPA3" : "Petroleo_gas_biocombustiveis",
                     "VBBR3" : "Petroleo_gas_biocombustiveis",

                     "FLRY3" : "Saude", "HAPV3" : "Saude", "HYPE3" : "Saude", "QUAL3" : "Saude", "RADL3" : "Saude", "RDOR3" : "Saude",

                     "CMIG4" : "Utilidade_publica", "CPFE3" : "Utilidade_publica", "CPLE6" : "Utilidade_publica", "EGIE3" : "Utilidade_publica",
                     "ELET3" : "Utilidade_publica", "ELET6" : "Utilidade_publica", "ENBR3" : "Utilidade_publica", "ENEV3" : "Utilidade_publica",
                     "ENGI11" : "Utilidade_publica", "EQTL3" : "Utilidade_publica", "SBSP3" : "Utilidade_publica", "TAEE11" : "Utilidade_publica"}

In [3]:
lista_ativos_setor =  [i + ".SA" for i in list(ativos_setor_IBOV.keys())]

In [ ]:
ativos = yf.download(lista_ativos_setor, start='2022-01-01', end = '2022-12-31', actions=True)[['Close', 'Dividends']]

In [5]:
soma_div = ativos['Dividends'].sum()

In [6]:
close_ativo = ativos['Close'].iloc[-1]

In [27]:
dy_2022 = pd.DataFrame(round(soma_div/close_ativo*100, 3), columns=['DY%']).sort_index()
dy_2022.index = dy_2022.index.map(lambda x :  x.replace('.SA', ""))


setores = pd.DataFrame(ativos_setor_IBOV.values(), index=ativos_setor_IBOV.keys(), columns=['Setores']).sort_index().rename_axis("Ticker")

dy_por_setor = pd.concat([dy_2022, setores], axis=1)
dy_por_setor.dropna(inplace=True)

### 6.2 Teste T de Student

*Exemplo: existe diferença significativa entre os setores "Utilidade Pública" e "Financeiro e outros" do IBOV quanto à média de Dividend Yield (%) para 2022?*

* Hipótese H0 (nula): As médias de DY não diferem entre os dois setores
* Hipótese H1 (alternativa): As médias são diferentes estatisticamente

##### **Testar a premissa de que os dados de cada grupo são normais**

*Lembrando que no teste de normalidade a hipótese H0 é de que os dados são normais, ou seja, buscaremos um p>0.05 para não rejeitar H0*

In [28]:
teste_t = dy_por_setor[(dy_por_setor['Setores'] == 'Utilidade_publica') | (dy_por_setor['Setores'] == 'Financeiro_e_outros')]

In [ ]:
dados_up = teste_t[(teste_t['Setores'] == 'Utilidade_publica')]['DY%']
teste_normalidade_g1 = stats.normaltest(dados_up)
teste_normalidade_g1

In [ ]:
dados_fo = teste_t[(teste_t['Setores'] == 'Financeiro_e_outros')]['DY%']
teste_normalidade_g2 = stats.normaltest(dados_fo)
teste_normalidade_g2

In [ ]:
teste_t_mean_std = pd.pivot_table(teste_t, index=['Setores'], values=['DY%'], aggfunc=[np.mean, np.std]).droplevel(1, axis=1)
teste_t_mean_std

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(name='Média e Desvio', x=teste_t_mean_std.index, y=teste_t_mean_std['mean'],
                     error_y=dict(type='data', array=teste_t_mean_std['std']),
                     marker_color='red', orientation='v'))

fig.update_layout(title_text='<b>Dividend Yield (média \xB1 erro padrão da média) <br> em 2022 entre dois setores do IBOV', xaxis_title= '<b>Setor IBOV',
                  yaxis_title= '<b>Dividend Yield (%)', template="none", margin=dict(l=75, r=20, t=100, b=50),
                  paper_bgcolor="#f7f8fa", font=dict(family="Arial", size=15, color="black"), width=600, height=400)

fig.show()